# ifctruss.solver

> All solvers that are supported

In [ ]:
# | default_exp solver

In [ ]:
# | export

# Copyright © 2023-2024  IfcTruss Contributors
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public License
# along with this program.  If not, see <https://www.gnu.org/licenses/>.

In [ ]:
# | hide
import nbdev
import nbdev.showdoc

In [ ]:
# | export
from collections import namedtuple
from typing import NamedTuple
import pandas as pd

In [ ]:
# | export
import ifctruss._direct_stiffness_method
import ifctruss._calfem

## Solver

In [ ]:
# | export
def direct_stiffness_method(
    nodes: pd.DataFrame,  # Pandas DataFrame with information regarding IfcStructuralPointConnection's
    bars: pd.DataFrame,  # Pandas DataFrame with information regarding IfcStructuralCurveMember's
    point_loads: pd.DataFrame,  # Pandas DataFrame with information regarding IfcStructuralLoadSingleForce's
) -> NamedTuple:  # NamedTuple with Pandas DataFrame's and str's
    System = ifctruss._direct_stiffness_method.DirectStiffnessMethod(
        bars=bars,
        nodes=nodes,
        point_loads=point_loads,
    )
    System.extend_nodes_df()
    System.extend_bars_df()
    System.create_element_stiffness_matrices()
    System.create_system_stiffness_matrice()
    System.create_force_vector()
    System.create_reduced_system_stiffness_matrice_and_reduced_force_vector()
    System.calculate_displacement_vector()

    System.calculate_force_vector()
    System.correct_force_vector()

    System.create_internal_force_matrices()
    System.calculate_normal_force()

    System.create_displacment_df()
    displacments = System.displacment_df

    System.create_force_df()
    forces = System.force_df

    System.create_normal_force_df()
    normal_forces = System.normal_force_df

    results = namedtuple(
        "df", "displacments forces normal_forces theory_type is_linear"
    )
    theory_type = "FIRST_ORDER_THEORY"
    is_linear = True
    return results(
        displacments, forces, normal_forces, theory_type, is_linear
    )

#### Example

In [ ]:
nodes = pd.DataFrame(
    {
        "Node": pd.Series([2, 1, 3, 4], dtype=int),
        "Coordinate_X": pd.Series([0, 0, -4e3, -4e3], dtype=float),
        "Coordinate_Y": pd.Series([0, 0, 0, 0], dtype=float),
        "Coordinate_Z": pd.Series([3e3, 0, 3e3, 6e3], dtype=float),
        "Translational_X": pd.Series([0, 1, 1, 1], dtype=bool),
        "Translational_Y": pd.Series([1, 1, 1, 1], dtype=bool),
        "Translational_Z": pd.Series([0, 1, 1, 1], dtype=bool),
    }
)

nodes

,Node,Coordinate_X,Coordinate_Y,Coordinate_Z,Translational_X,Translational_Y,Translational_Z
0,2,0.0,0.0,3000.0,False,True,False
1,1,0.0,0.0,0.0,True,True,True
2,3,-4000.0,0.0,3000.0,True,True,True
3,4,-4000.0,0.0,6000.0,True,True,True


In [ ]:
bars = pd.DataFrame(
    {
        "Bar": pd.Series([1, 2, 3], dtype=int),
        "Start_node": pd.Series([2, 2, 2], dtype=int),
        "End_node": pd.Series([1, 3, 4], dtype=int),
        "Cross-sectional_area": pd.Series([1e3, 1e3, 1e3], dtype=float),
        "Modulus_of_elasticity": pd.Series([1e3, 1e3, 1e3], dtype=float),
    }
)

bars

,Bar,Start_node,End_node,Cross-sectional_area,Modulus_of_elasticity
0,1,2,1,1000.0,1000.0
1,2,2,3,1000.0,1000.0
2,3,2,4,1000.0,1000.0


In [ ]:
point_loads = pd.DataFrame(
    {
        # fmt: off
    "Point_Load": pd.Series([1,], dtype=int,),
    "Node": pd.Series([2,], dtype=int,),
    "Force_X": pd.Series([100e3,], dtype=float,),
    "Force_Y": pd.Series([0,], dtype=float,),
    "Force_Z": pd.Series([-100e3,], dtype=float,),
        # fmt: on
    }
)
point_loads

,Point_Load,Node,Force_X,Force_Y,Force_Z
0,1,2,100000.0,0.0,-100000.0


In [ ]:
results = direct_stiffness_method(
    nodes=nodes, bars=bars, point_loads=point_loads
)

In [ ]:
results.displacments

,Node,Displacement_X,Displacement_Y,Displacement_Z
0,2,214.814815,0.0,-195.833333


In [ ]:
results.forces

,Node,Force_X,Force_Y,Force_Z
1,1,0.000000,0.0,65277.777778
2,3,-53703.703704,0.0,0.000000
3,4,-46296.296296,0.0,34722.222222


In [ ]:
(
    displacments,
    forces,
    normal_forces,
    theory_type,
    is_linear,
) = direct_stiffness_method(nodes=nodes, bars=bars, point_loads=point_loads)

In [ ]:
normal_forces.style

,Bar,Normal_force,Type_of_normal_force
0,1,-65277.777778,Compressive force
1,2,53703.703704,Tensile force
2,3,57870.370370,Tensile force


In [ ]:
theory_type

'FIRST_ORDER_THEORY'

In [ ]:
is_linear

True

In [ ]:
# | export
def calfem(
    nodes: pd.DataFrame,  # Pandas DataFrame with information regarding IfcStructuralPointConnection's
    bars: pd.DataFrame,  # Pandas DataFrame with information regarding IfcStructuralCurveMember's
    point_loads: pd.DataFrame,  # Pandas DataFrame with information regarding IfcStructuralLoadSingleForce's
) -> NamedTuple:  # NamedTuple with Pandas DataFrame's and str's
    System = ifctruss._calfem.CALFEM(
        bars=bars,
        nodes=nodes,
        point_loads=point_loads,
    )

    System.extend_nodes_df()
    System.extend_bars_df()
    System.create_element_stiffness_matrices()
    System.create_system_stiffness_matrice()
    System.create_force_vector()
    System.calculate_displacement_vector_and_calculate_force_vector()

    System.correct_force_vector()

    System.calculate_normal_force()

    System.create_displacment_df()
    displacments = System.displacment_df

    System.create_force_df()
    forces = System.force_df

    System.create_normal_force_df()
    normal_forces = System.normal_force_df

    results = namedtuple(
        "df", "displacments forces normal_forces theory_type is_linear"
    )
    theory_type = "FIRST_ORDER_THEORY"
    is_linear = True
    return results(
        displacments, forces, normal_forces, theory_type, is_linear
    )

#### Example

In [ ]:
results_calfem = calfem(nodes=nodes, bars=bars, point_loads=point_loads)

In [ ]:
results_calfem.displacments

,Node,Displacement_X,Displacement_Y,Displacement_Z
0,2,214.814815,0.0,-195.833333


In [ ]:
results_calfem.forces

,Node,Force_X,Force_Y,Force_Z
1,1,0.000000,0.0,65277.777778
2,3,-53703.703704,0.0,0.000000
3,4,-46296.296296,0.0,34722.222222


In [ ]:
results_calfem.normal_forces

,Bar,Normal_force,Type_of_normal_force
0,1,-65277.777778,Compressive force
1,2,53703.703704,Tensile force
2,3,57870.370370,Tensile force


## Comparison
### Resulate

Test whether `calfem` calculates the same results as `direct_stiffness_method`:

In [ ]:
import numpy as np

In [ ]:
assert np.all(
    np.isclose(
        results.normal_forces["Normal_force"],
        results_calfem.normal_forces["Normal_force"],
    )
)

In [ ]:
assert np.all(
    np.isclose(results.forces["Force_X"], results_calfem.forces["Force_X"])
)

In [ ]:
assert np.all(
    np.isclose(results.forces["Force_Y"], results_calfem.forces["Force_Y"])
)

In [ ]:
assert np.all(
    np.isclose(results.forces["Force_Z"], results_calfem.forces["Force_Z"])
)

In [ ]:
assert np.all(
    np.isclose(
        results.displacments["Displacement_X"],
        results_calfem.displacments["Displacement_X"],
    )
)

In [ ]:
assert np.all(
    np.isclose(
        results.displacments["Displacement_Y"],
        results_calfem.displacments["Displacement_Y"],
    )
)

In [ ]:
assert np.all(
    np.isclose(
        results.displacments["Displacement_Z"],
        results_calfem.displacments["Displacement_Z"],
    )
)

### Speed

Compare the speed of `direct_stiffness_method`and `calfem`

In [ ]:
import timeit
import statistics

In [ ]:
execution_times = timeit.repeat(
    lambda: direct_stiffness_method(
        nodes=nodes, bars=bars, point_loads=point_loads
    ),
    number=200,
    repeat=7,
)

mean_time = statistics.mean(execution_times)
stdev_time = statistics.stdev(execution_times)

print("Mean execution time:", mean_time, "seconds")
print("Standard deviation:", stdev_time, "seconds")

Mean execution time: 2.5156487490000603 seconds
Standard deviation: 0.3370494115779167 seconds


In [ ]:
execution_times = timeit.repeat(
    lambda: calfem(nodes=nodes, bars=bars, point_loads=point_loads),
    number=200,
    repeat=7,
)

mean_time = statistics.mean(execution_times)
stdev_time = statistics.stdev(execution_times)

print("Mean execution time:", mean_time, "seconds")
print("Standard deviation:", stdev_time, "seconds")

Mean execution time: 3.4454708377145704 seconds
Standard deviation: 0.19648677400024356 seconds


In [ ]:
# | hide
nbdev.nbdev_export()